In [10]:
import torch
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
from adversarial_dataset import AdversarialDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torchvision import models
import torch.nn.functional as F
from tqdm.notebook import tqdm
import os
import numpy as np

In [2]:
class TensorToDevice(object):
    def __init__(self, device):
        self.device = device

    def __call__(self, image):
        image = image.to(self.device)
        return image

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

transform = transforms.Compose([
    transforms.ToTensor(),
    TensorToDevice(device),
    transforms.Normalize(mean=mean, std=std)
])

noise_transform = transforms.Compose([
    transforms.ToTensor()
])

In [5]:
dataset = AdversarialDataset(annotation_file="images.csv",
                             categories_file="categories.csv",
                             img_dir="images",
                             img_transform=transform,
                             noise_transform=noise_transform)

dataloader = DataLoader(dataset, batch_size=8, shuffle=False)

model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
model_to_train = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
model, model_to_train = model.to(device), model_to_train.to(device)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [16]:
def fgsm(input, labels, model, epsilon):
    inputs.requires_grad = True
    output = model(inputs)
    loss = F.nll_loss(output, labels)

    model.zero_grad()
    loss.backward()
    return epsilon*inputs.grad.data.sign()

In [25]:
train, test = train_test_split(np.arange(1000), shuffle=True, test_size=.2)

train_split = Subset(dataset, train)
test_split = Subset(dataset, test)

train_loader = DataLoader(train_split, batch_size=16)
test_loader = DataLoader(test_split, batch_size=16)

n_epochs = 10
model.eval()
model_to_train.train()
epsilon = .1

train_loss_history, test_loss_history = [], []
train_accuracy_history, test_accuracy_history = [], []

for i in range(n_epochs):
    print(f"Epoch: {i + 1}/{n_epochs}")
    model_to_train.train()
    
    running_loss_train, running_loss_test = 0.0, 0.0
    running_corrects_train, running_corrects_test = 0, 0
    print("Training...")
    for inputs, _, labels, _, id in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        noise = fgsm(inputs, labels, model, .1)
        adversarial_image = inputs + noise
        assert adversarial_image.shape == inputs.shape
        
        output, _ = model_to_train(adversarial_image)
        train_loss = loss_function(output, labels)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        _, preds = torch.max(output, dim=1)
        
        running_loss_train += train_loss.item() * inputs.size(0)
        running_corrects_train += torch.sum(preds == labels)
        
    model_to_train.eval()
    print("Evaluating...")
    for inputs, _, labels, _, id in tqdm(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        noise = fgsm(inputs, labels, model, .1)
        adversarial_image = inputs + noise
        
        assert adversarial_image.shape == inputs.shape
        
        output = model_to_train(adversarial_image)
        test_loss = loss_function(output, labels)
        _, preds = torch.max(output, dim=1)
        
        running_loss_test += test_loss.item() * inputs.size(0)
        running_corrects_test += torch.sum(preds == labels)
        
    train_accuracy_history.append(running_corrects_train/len(train_split))
    test_accuracy_history.append(running_corrects_test/len(test_split))
    
    train_loss_history.append(running_loss_train/len(train_split))
    test_loss_history.append(running_loss_test/len(test_split))
    
    print(f"Train accuracy: {round(train_accuracy_history[-1].item(), 2)}. Test accuracy: {round(test_accuracy_history[-1].item(), 2)}")
    print(f"Train loss: {round(train_loss_history[-1], 2)}. Test loss: {round(test_loss_history[-1], 2)}")
    print("-------------------------------------------------")


Epoch: 1/10
Training...


  0%|          | 0/50 [00:00<?, ?it/s]

Evaluating...


  0%|          | 0/13 [00:00<?, ?it/s]

Train accuracy: 0.26. Test accuracy: 0.29
Train loss: 4.11. Test loss: 4.31
-------------------------------------------------
Epoch: 2/10
Training...


...

Train accuracy: 0.29. Test accuracy: 0.29
Train loss: 4.03. Test loss: 4.31
-------------------------------------------------
Epoch: 10/10
Training...


  0%|          | 0/50 [00:00<?, ?it/s]

Evaluating...


  0%|          | 0/13 [00:00<?, ?it/s]

Train accuracy: 0.27. Test accuracy: 0.29
Train loss: 4.07. Test loss: 4.31
-------------------------------------------------


In [23]:
train_accuracy_history[-1].item()

0.25999999046325684